# Generalised Linear Regression

In [12]:
%%writefile "requirements.txt"
pandas==2
numpy==1.26
scipy==1.12

Overwriting requirements.txt


In [13]:
!pip install -r "requirements.txt"

In [14]:
import numpy as np
import pandas as pd
import scipy.optimize as sci
from scipy.special import factorial

## Maximum Likelihood for the Poisson GLM

The log likelihood for the Poisson GLM is

\begin{equation}
    \log l(y|\beta_0,\beta_1,\dots,\beta_p,x) = \sum_{i=1}^n[-\lambda(x_i)+y_i\log(\lambda(x_i))-\log(y_i!)],
\end{equation}

where $x$ is the predictor, $y$ the observation and 

\begin{equation}
    \log(\lambda(x_i)) = \beta_0 + \beta_1x_{i1} + \dots + \beta_px_{ip}.
\end{equation}

In [15]:
def poisson_logl(b, x, y):
    """Compute the log likelihood of the poisson distribution.

    Args:
        b (ndarray): contains all the beta parameters
        x (ndarray): contains the predictor values
        y (ndarray): contains the observation values

    Returns:
        int : log likelihood
    """
    aux = b[0] + b[1:] * x
    
    return - np.sum(y * aux - np.log(factorial(y)) - np.exp(aux)) # Minus to compute the maximum

def poisson_pred(b, x):
    """Draw samples from a Poisson distribution determined by the parameter lmb

    Args:
        b (ndarray): contains all the beta parameters
        x (ndarray): contains the predictor values

    Returns:
        ndarray: Drawns sampled from the parametrized Poisson distribution
    """
    lmb = b[0] + b[1:] * x
    
    return np.random.poisson(lmb, size = (3, lmb.shape[0]))

Load the data and normalized the predictor

In [16]:
data = pd.read_csv("bird_count.csv")
x = data["yr"].to_numpy() - np.min(data["yr"].to_numpy()) # Predictor
y = data["count"].to_numpy() # Observation 

Find the minimum of the function `poisson_logl` using `sicpy.optimize.minimize()`

In [17]:
res = sci.minimize(poisson_logl, np.full(2, 1), (x, y))
b = res.x # Optimized parameters

# Print the optimal parameters
print("Optimized parameters: ", b)

Optimized parameters:  [ 2.32533192 -0.03244318]


Generate samples of data

In [18]:
pred = poisson_pred(b, x) # Predictions

results = np.vstack((x + np.min(data["yr"].to_numpy()), pred))
print("Predicted values")
print(results)

Predicted values
[[2011 2010 2002 2006 2008 2012 2000 2005 2007 2004 1999 2001 2009]
 [   1    2    5    2    2    2    0    3    1    3    0    2    3]
 [   2    2    3    0    1    1    1    3    2    5    4    3    2]
 [   3    2    1    4    3    3    1    1    2    7    1    0    2]]


# FAIR data

This workflow relates to FAIR data principles because
*   It is uploaded in Github with keywords, so it is findable by both humans and computers
*   It is also accessible, since it is contained in a Github repository
*   The data is contained in a .csv file, so it is interoperable
*   There is a data usage license and the keywords describe accurate and relevant attributes, so it is reusable